In [15]:
%%capture
pip install -r requirements.txt


In [1]:
# Bank Simulation Notebook with Double-Entry Bookkeeping

# Import Necessary Modules
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Initialize Database and Tables
# Connect to SQLite database
conn = sqlite3.connect('bank_simulation.db')
cursor = conn.cursor()

# Create clients table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS clients (
        client_id INTEGER PRIMARY KEY,
        name TEXT,
        account_type TEXT,
        currency TEXT,
        balance REAL
    )
''')

# Create ledger_entries table for double-entry bookkeeping
cursor.execute('''
    CREATE TABLE IF NOT EXISTS ledger_entries (
        entry_id INTEGER PRIMARY KEY AUTOINCREMENT,
        date TEXT,
        transaction_id INTEGER,
        account TEXT,
        debit REAL,
        credit REAL,
        description TEXT,
        FOREIGN KEY(transaction_id) REFERENCES transactions(transaction_id)
    )
''')

# Create transactions table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS transactions (
        transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
        date TEXT,
        client_id INTEGER,
        type TEXT,
        amount REAL,
        currency TEXT,
        FOREIGN KEY(client_id) REFERENCES clients(client_id)
    )
''')

conn.commit()

# Create Mock Clients
# Sample client data
clients = [
    (1, 'Alice Rossi', 'savings', 'EUR', 10000.00),
    (2, 'Bob Verdi', 'checking', 'USD', 5000.00),
    (3, 'Carla Bianchi', 'corporate', 'EUR', 25000.00),
    (4, 'David Nero', 'savings', 'GBP', 8000.00),
    (5, 'Eva Blu', 'checking', 'EUR', 12000.00),
    # Add more clients as needed
]

# Insert clients into the database
cursor.executemany('''
    INSERT OR REPLACE INTO clients (client_id, name, account_type, currency, balance)
    VALUES (?, ?, ?, ?, ?)
''', clients)

conn.commit()

# Define Transaction Functions with Double-Entry Bookkeeping
def deposit(client_id, amount, currency, date):
    # Update client's balance
    cursor.execute('''
        UPDATE clients SET balance = balance + ?
        WHERE client_id = ? AND currency = ?
    ''', (amount, client_id, currency))
    
    # Record the transaction
    cursor.execute('''
        INSERT INTO transactions (date, client_id, type, amount, currency)
        VALUES (?, ?, 'deposit', ?, ?)
    ''', (date, client_id, amount, currency))
    transaction_id = cursor.lastrowid
    
    # Double-entry ledger entries
    # Debit Cash/Bank Account
    record_ledger_entry(date, transaction_id, 'Cash/Bank', amount, 0, f'Deposit from Client {client_id}')
    # Credit Client's Account
    record_ledger_entry(date, transaction_id, f'Client {client_id} Account', 0, amount, f'Deposit to Client {client_id} Account')
    conn.commit()

def withdraw(client_id, amount, currency, date):
    # Update client's balance
    cursor.execute('''
        UPDATE clients SET balance = balance - ?
        WHERE client_id = ? AND currency = ?
    ''', (amount, client_id, currency))
    
    # Record the transaction
    cursor.execute('''
        INSERT INTO transactions (date, client_id, type, amount, currency)
        VALUES (?, ?, 'withdrawal', ?, ?)
    ''', (date, client_id, -amount, currency))
    transaction_id = cursor.lastrowid
    
    # Double-entry ledger entries
    # Debit Client's Account
    record_ledger_entry(date, transaction_id, f'Client {client_id} Account', amount, 0, f'Withdrawal from Client {client_id} Account')
    # Credit Cash/Bank Account
    record_ledger_entry(date, transaction_id, 'Cash/Bank', 0, amount, f'Withdrawal to Client {client_id}')
    conn.commit()

def apply_fee(client_id, amount, currency, date):
    # Update client's balance
    cursor.execute('''
        UPDATE clients SET balance = balance - ?
        WHERE client_id = ? AND currency = ?
    ''', (amount, client_id, currency))
    
    # Record the transaction
    cursor.execute('''
        INSERT INTO transactions (date, client_id, type, amount, currency)
        VALUES (?, ?, 'fee', ?, ?)
    ''', (date, client_id, -amount, currency))
    transaction_id = cursor.lastrowid
    
    # Double-entry ledger entries
    # Debit Client's Account
    record_ledger_entry(date, transaction_id, f'Client {client_id} Account', amount, 0, f'Fee Charged to Client {client_id}')
    # Credit Fee Income Account
    record_ledger_entry(date, transaction_id, 'Fee Income', 0, amount, f'Fee Income from Client {client_id}')
    conn.commit()

def loan_payment(client_id, amount, currency, date):
    # Update client's balance
    cursor.execute('''
        UPDATE clients SET balance = balance - ?
        WHERE client_id = ? AND currency = ?
    ''', (amount, client_id, currency))
    
    # Record the transaction
    cursor.execute('''
        INSERT INTO transactions (date, client_id, type, amount, currency)
        VALUES (?, ?, 'loan_payment', ?, ?)
    ''', (date, client_id, -amount, currency))
    transaction_id = cursor.lastrowid
    
    # Double-entry ledger entries
    # Debit Client's Account
    record_ledger_entry(date, transaction_id, f'Client {client_id} Account', amount, 0, f'Loan Payment from Client {client_id}')
    # Credit Loan Receivable
    record_ledger_entry(date, transaction_id, 'Loan Receivable', 0, amount, f'Loan Payment Received from Client {client_id}')
    conn.commit()

def fx_transaction(client_id, amount, from_currency, to_currency, date):
    # Simplified FX rate
    fx_rates = {
        ('EUR', 'USD'): 1.1,
        ('USD', 'EUR'): 0.9,
        ('EUR', 'GBP'): 0.85,
        ('GBP', 'EUR'): 1.15,
        ('USD', 'GBP'): 0.77,
        ('GBP', 'USD'): 1.3
    }
    rate = fx_rates.get((from_currency, to_currency), 1)
    converted_amount = amount * rate
    
    # Update client's balances
    # Deduct from original currency
    cursor.execute('''
        UPDATE clients SET balance = balance - ?
        WHERE client_id = ? AND currency = ?
    ''', (amount, client_id, from_currency))
    # Add to new currency
    cursor.execute('''
        UPDATE clients SET balance = balance + ?
        WHERE client_id = ? AND currency = ?
    ''', (converted_amount, client_id, to_currency))
    
    # Record the transaction
    cursor.execute('''
        INSERT INTO transactions (date, client_id, type, amount, currency)
        VALUES (?, ?, 'fx_transaction', ?, ?)
    ''', (date, client_id, -amount, from_currency))
    transaction_id = cursor.lastrowid
    
    # Double-entry ledger entries
    # Debit FX Loss/Gain Account (simplified)
    record_ledger_entry(date, transaction_id, 'FX Gain/Loss', amount - converted_amount, 0, f'FX Transaction Loss/Gain for Client {client_id}')
    # Credit Client's Account in From Currency
    record_ledger_entry(date, transaction_id, f'Client {client_id} Account ({from_currency})', 0, amount, f'FX Transaction Deduct from Client {client_id} Account')
    # Debit Client's Account in To Currency
    record_ledger_entry(date, transaction_id, f'Client {client_id} Account ({to_currency})', converted_amount, 0, f'FX Transaction Add to Client {client_id} Account')
    # Credit FX Transactions
    record_ledger_entry(date, transaction_id, 'FX Transactions', 0, converted_amount, f'FX Transaction Completed for Client {client_id}')
    conn.commit()

def record_ledger_entry(date, transaction_id, account, debit, credit, description):
    cursor.execute('''
        INSERT INTO ledger_entries (date, transaction_id, account, debit, credit, description)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (date, transaction_id, account, debit, credit, description))

# Simulate One Day of Transactions
def simulate_one_day(date):
    # Get client_ids and currencies from the database
    cursor.execute('SELECT client_id, currency FROM clients')
    clients_data = cursor.fetchall()
    for client_id, client_currency in clients_data:
        # Random transaction
        txn_type = np.random.choice(['deposit', 'withdrawal', 'fee', 'loan_payment', 'fx_transaction'])
        amount = round(np.random.uniform(100, 1000), 2)
        if txn_type == 'deposit':
            deposit(client_id, amount, client_currency, date)
        elif txn_type == 'withdrawal':
            withdraw(client_id, amount, client_currency, date)
        elif txn_type == 'fee':
            fee_amount = round(amount * 0.01, 2)  # 1% fee
            apply_fee(client_id, fee_amount, client_currency, date)
        elif txn_type == 'loan_payment':
            loan_payment(client_id, amount, client_currency, date)
        elif txn_type == 'fx_transaction':
            # Choose a different currency for FX
            currencies = ['EUR', 'USD', 'GBP']
            currencies.remove(client_currency)
            to_currency = np.random.choice(currencies)
            fx_transaction(client_id, amount, client_currency, to_currency, date)

# Generate Daily Ledger
def generate_daily_ledger(date):
    df = pd.read_sql_query('''
        SELECT * FROM ledger_entries WHERE date = ?
    ''', conn, params=(date,))
    df.to_csv(f'daily_ledger_{date}.csv', index=False)
    total_debits = df['debit'].sum()
    total_credits = df['credit'].sum()
    print(f'Daily ledger for {date} generated with {len(df)} entries.')
    print(f'Total Debits: {total_debits}, Total Credits: {total_credits}')
    if round(total_debits, 2) == round(total_credits, 2):
        print('The ledger is balanced.')
    else:
        print('The ledger is NOT balanced.')

# Simulate One Month
def simulate_one_month():
    start_date = datetime.now() - timedelta(days=29)
    for i in range(30):
        current_date = (start_date + timedelta(days=i)).strftime('%Y-%m-%d')
        simulate_one_day(current_date)
        generate_daily_ledger(current_date)

simulate_one_month()

# Generate Monthly Financial Statements
def generate_monthly_reports():
    # Ledger entries for the month
    df_ledger = pd.read_sql_query('SELECT * FROM ledger_entries', conn)
    
    # Check if the ledger balances
    total_debits = df_ledger['debit'].sum()
    total_credits = df_ledger['credit'].sum()
    print(f'Monthly Total Debits: {total_debits}, Total Credits: {total_credits}')
    if round(total_debits, 2) == round(total_credits, 2):
        print('The monthly ledger is balanced.')
    else:
        print('The monthly ledger is NOT balanced.')
    
    # Balance Sheet
    balances = pd.read_sql_query('SELECT currency, SUM(balance) as total_balance FROM clients GROUP BY currency', conn)
    display(balances)
    
    # Income Statement (simplified)
    income_accounts = ['Fee Income', 'FX Gain/Loss', 'Interest Income']
    expenses_accounts = ['Interest Expense', 'Operating Expenses']
    
    income_statement = df_ledger[df_ledger['account'].isin(income_accounts + expenses_accounts)]
    income_summary = income_statement.groupby('account').apply(lambda x: x['credit'].sum() - x['debit'].sum()).reset_index(name='Net Amount')
    display(income_summary)
    
    # Save to CSV
    balances.to_csv('monthly_balance_sheet.csv', index=False)
    income_summary.to_csv('monthly_income_statement.csv', index=False)
    print('Monthly financial statements generated.')

generate_monthly_reports()

# Calculate Regulatory Ratios
def calculate_regulatory_ratios():
    # Simplified calculations for demonstration
    total_assets = cursor.execute('SELECT SUM(balance) FROM clients').fetchone()[0]
    total_liabilities = total_assets * 0.8  # Assuming 80% liabilities
    capital = total_assets - total_liabilities
    car = (capital / total_assets) * 100

    ratios = {
        'Capital Adequacy Ratio (CAR)': f'{car:.2f}%',
        # Add LCR, NSFR calculations as needed
    }
    display(ratios)
    with open('regulatory_ratios.json', 'w') as f:
        json.dump(ratios, f)
    print('Regulatory ratios calculated.')

calculate_regulatory_ratios()

# Generate Regulatory Reports
def generate_regulatory_reports():
    # Suspicious Transactions
    df_transactions = pd.read_sql_query('SELECT * FROM transactions', conn)
    suspicious_transactions = df_transactions[df_transactions['amount'].abs() > 10000]
    display(suspicious_transactions)
    suspicious_transactions.to_csv('suspicious_transactions.csv', index=False)

    # Large Exposures
    large_exposures = df_transactions.groupby('client_id')['amount'].sum().reset_index()
    large_exposures = large_exposures[large_exposures['amount'].abs() > 50000]
    display(large_exposures)
    large_exposures.to_csv('large_exposures.csv', index=False)
    print('Regulatory reports generated.')

generate_regulatory_reports()

# Visualizations
# Transaction Types Distribution
plt.figure(figsize=(10,6))
sns.countplot(data=df_transactions, x='type')
plt.title('Transaction Types Distribution')
plt.xticks(rotation=45)
plt.show()

# Total Debit and Credit Amounts Over Time
df_ledger['date'] = pd.to_datetime(df_ledger['date'])
daily_totals = df_ledger.groupby('date').agg({'debit': 'sum', 'credit': 'sum'}).reset_index()
plt.figure(figsize=(14,7))
plt.plot(daily_totals['date'], daily_totals['debit'], label='Total Debits')
plt.plot(daily_totals['date'], daily_totals['credit'], label='Total Credits')
plt.title('Daily Total Debits and Credits')
plt.legend()
plt.show()

# Client Balances
df_clients = pd.read_sql_query('SELECT * FROM clients', conn)
plt.figure(figsize=(10,6))
sns.barplot(data=df_clients, x='name', y='balance', hue='currency')
plt.title('Client Balances')
plt.xticks(rotation=45)
plt.show()

# Close the database connection
conn.close()


Daily ledger for 2024-08-20 generated with 10 entries.
Total Debits: 1889.46, Total Credits: 1889.46
The ledger is balanced.
Daily ledger for 2024-08-21 generated with 10 entries.
Total Debits: 3079.1000000000004, Total Credits: 3079.1000000000004
The ledger is balanced.
Daily ledger for 2024-08-22 generated with 12 entries.
Total Debits: 2104.63, Total Credits: 2225.916
The ledger is NOT balanced.
Daily ledger for 2024-08-23 generated with 16 entries.
Total Debits: 2024.94, Total Credits: 3308.8075
The ledger is NOT balanced.
Daily ledger for 2024-08-24 generated with 12 entries.
Total Debits: 1168.5600000000002, Total Credits: 1436.072
The ledger is NOT balanced.
Daily ledger for 2024-08-25 generated with 12 entries.
Total Debits: 1197.45, Total Credits: 1435.996
The ledger is NOT balanced.
Daily ledger for 2024-08-26 generated with 16 entries.
Total Debits: 2042.47, Total Credits: 3473.313
The ledger is NOT balanced.
Daily ledger for 2024-08-27 generated with 16 entries.
Total Debit

,currency,total_balance
0,EUR,27382.63
1,GBP,6914.27
2,USD,-3280.03


C:\Users\wjbea\AppData\Local\Temp\ipykernel_89268\2554287299.py:284: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  income_summary = income_statement.groupby('account').apply(lambda x: x['credit'].sum() - x['debit'].sum()).reset_index(name='Net Amount')


,account,Net Amount
0,FX Gain/Loss,-812.4018
1,Fee Income,204.9500


Monthly financial statements generated.


...

,transaction_id,date,client_id,type,amount,currency


,client_id,amount


Regulatory reports generated.


NameError: name 'df_transactions' is not defined

<Figure size 1000x600 with 0 Axes>